# 4 Modelo: seleccionar el mejor Cluster

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('precision', 4)
pd.set_option('display.max_rows', 100)

In [3]:
df = pd.read_csv('data_preparada.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22552 entries, 0 to 22551
Data columns (total 77 columns):
name                                22552 non-null object
experiences_offered                 22552 non-null object
host_since                          22526 non-null object
host_response_time                  22552 non-null object
host_response_rate                  22552 non-null float64
host_acceptance_rate                22552 non-null float64
host_is_superhost                   22552 non-null float64
host_total_listings_count           22552 non-null float64
host_has_profile_pic                22552 non-null float64
host_identity_verified              22552 non-null float64
neighbourhood_cleansed              22552 non-null object
neighbourhood_group_cleansed        22552 non-null object
city                                22552 non-null object
state                               22552 non-null object
country                             22552 non-null object
latitude       

In [4]:
# set variables tipo fecha
date_columns = [
                "first_review",
                "last_review",
                "host_since",
               ]

for column in date_columns:
    df[column] = pd.to_datetime(df[column],errors='coerce')

In [5]:
#seleccionar columnas para transformar

exclude_columns = []

for column in df.columns:
    
    if df[column].dtype == "int" or df[column].dtype == "float":
        pass
    elif df[column].dtype == (object):
        exclude_columns.append(column)
    elif df[column].dtype == "bool":
        pass
    else :
        exclude_columns.append(column)
exclude_columns

['name',
 'experiences_offered',
 'host_since',
 'host_response_time',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'city',
 'state',
 'country',
 'property_type',
 'room_type',
 'bed_type',
 'calendar_updated',
 'first_review',
 'last_review',
 'cancellation_policy']

In [6]:
# Label Encoder
from sklearn.preprocessing import LabelEncoder

if False:
    for column in df.columns:
        if df[column].dtype == (object):
            encoder = LabelEncoder()
            try:
                df[column + '_to_cluster'] = encoder.fit_transform(df[column])
            except TypeError as t_err:
                try:
                    df.drop(columns=[column + '_to_cluster'],inplace=True)
                except KeyError as k_err:
                    print("Ignorado " + column+" - KeyError: "+str(k_err))
                print("Ignorado " + column+" - TypeError: "+str(t_err))


In [7]:
#Feature selector that removes all low-variance features.

df_colums_selected = []

for column in df.columns:
    if column not in exclude_columns:
        try:
            varianza = df[column].var()
        except KeyError:
            varianza = 0.0
        
        if  varianza > 0.3:
            df_colums_selected.append(column)
        else:
            print ("Variables exluidas: " + column)

Variables exluidas: host_acceptance_rate
Variables exluidas: host_is_superhost
Variables exluidas: host_has_profile_pic
Variables exluidas: host_identity_verified
Variables exluidas: latitude
Variables exluidas: longitude
Variables exluidas: is_location_exact
Variables exluidas: bathrooms
Variables exluidas: has_availability
Variables exluidas: requires_license
Variables exluidas: jurisdiction_names
Variables exluidas: instant_bookable
Variables exluidas: is_business_travel_ready
Variables exluidas: require_guest_profile_picture
Variables exluidas: require_guest_phone_verification
Variables exluidas: has_a_summary
Variables exluidas: has_a_space
Variables exluidas: has_a_description
Variables exluidas: has_a_neighborhood_overview
Variables exluidas: has_a_notes
Variables exluidas: has_a_transit
Variables exluidas: has_a_access
Variables exluidas: has_a_interaction
Variables exluidas: has_a_house_rules
Variables exluidas: has_a_host_about
Variables exluidas: has_a_license


In [8]:
df.fillna(0,inplace=True)

In [9]:
# segmentacion por Kmeans

from sklearn.cluster import *
from sklearn import metrics
import json
results = []
if True:
    number_clusters = range(3,15)
    for k_clusters in number_clusters:
        print ("____________________________________")
        print (" Modelo Kmedias")
        model = MiniBatchKMeans(n_clusters=k_clusters, n_init=200, random_state=0)
        cluster_fit = model.fit(df[df_colums_selected])
        predict = cluster_fit.predict(df[df_colums_selected])
        df_y = pd.DataFrame(predict)
        df["TEMP_CLUSTER_"+str(k_clusters)] = pd.Series(predict, index=df.index)
        df["TEMP_CLUSTER_"+str(k_clusters)] = df["TEMP_CLUSTER_"+str(k_clusters)] + 1
        table_df = pd.DataFrame(df_y.groupby(0).size())[0]
        table = json.loads(table_df.to_json(orient='columns'))
        print(table_df)
        #table_df.plot.pie(y='"KMEDIAS_CLUSTER_"+str(k_clusters)', figsize=(5, 5))

            # The Silhouette Coefficient is calculated using the mean intra-cluster distance (a) 
            # and the mean nearest-cluster distance (b) for each sample. 
            # The Silhouette Coefficient for a sample is (b - a) / max(a, b). 
            # To clarify, b is the distance between a sample and the nearest cluster that the sample is not a part of. 
            # Note that Silhouette Coefficient is only defined if number of labels is 2 <= n_labels <= n_samples - 1.
            # This function returns the mean Silhouette Coefficient over all samples. 
            # The best value is 1 and the worst value is -1. Values near 0 indicate overlapping clusters.
            # Negative values generally indicate that a sample has been assigned to the wrong cluster, 
            # as a different cluster is more similar.
        sample_size=int(len(df_y)/2)

        silhouette=metrics.silhouette_score(df[df_colums_selected], predict,sample_size=sample_size ,random_state=0)
        print("Silhouette Coefficient: %0.3f" % silhouette )

            # The score is defined as the average similarity measure of each cluster with its most similar cluster, 
            # where similarity is the ratio of within-cluster distances to between-cluster distances. 
            # Thus, clusters which are farther apart and less dispersed will result in a better score
            # The minimum score is zero, with lower values indicating better clustering.
        davies_bouldin_score =  metrics.davies_bouldin_score(df[df_colums_selected], predict)
        print("Davies-Bouldin score: %0.3f" % davies_bouldin_score)

            # The score is defined as ratio between the within-cluster dispersion and the between-cluster dispersion.
            # There is no "acceptable" cut-off value. You simply compare CH values by eye. The higher the value, 
            # the "better" is the solution. If on the line-plot of CH values there appears that one solution give 
            # a peak or at least an abrupt elbow, choose it. If, on the contrary, the line is smooth - horizontal
            # or ascending or descending - then there is no reason to prefer one solution to others.
        calinski_harabasz_score = metrics.calinski_harabasz_score(df[df_colums_selected], predict)
        print("Calinski and Harabasz score: %0.3f" % calinski_harabasz_score)

        
        for k,v in table.items():
            results.append( {
                            "method": "KMeans",
                            "n_clusters":k_clusters,
                            "silhouette_score":silhouette,
                            "davies_bouldin_score":davies_bouldin_score,
                            "calinski_harabasz_score":calinski_harabasz_score,
                            "cluster":k,
                            "num_columns":v,
                           })


____________________________________
 Modelo Kmedias
0
0    9533
1    3908
2    9111
Name: 0, dtype: int64
Silhouette Coefficient: 0.026
Davies-Bouldin score: 1.996
Calinski and Harabasz score: 1.752
____________________________________
 Modelo Kmedias
0
0        3
1     3907
2    16869
3     1773
Name: 0, dtype: int64
Silhouette Coefficient: 0.564
Davies-Bouldin score: 0.835
Calinski and Harabasz score: 42119.043
____________________________________
 Modelo Kmedias
0
0    8231
1    3898
2     146
3    8682
4    1595
Name: 0, dtype: int64
Silhouette Coefficient: 0.033
Davies-Bouldin score: 1.998
Calinski and Harabasz score: 1.024
____________________________________
 Modelo Kmedias
0
0    8242
1    1739
2    8648
3    1553
4    2163
5     207
Name: 0, dtype: int64
Silhouette Coefficient: 0.020
Davies-Bouldin score: 1.997
Calinski and Harabasz score: 0.824
____________________________________
 Modelo Kmedias
0
0    8180
1    1558
2    2146
3    8608
4    1727
5     161
6     172
Name: 0

In [10]:
#Listamos los mejores resultados
df_results = pd.DataFrame(results)
df_clusters_selected = df_results.loc[df_results['silhouette_score'] >= df_results.silhouette_score.max()-0.1]
df_clusters_selected= df_clusters_selected.groupby(["n_clusters"] ).first() 
df_clusters_selected.head(100)

,method,silhouette_score,davies_bouldin_score,calinski_harabasz_score,cluster,num_columns
n_clusters,,,,,,
4,KMeans,0.5640,0.8350,42119.0429,0,3
9,KMeans,0.6185,1.9929,7.8710,0,1725
10,KMeans,0.6080,1.7497,296.9758,0,8198
12,KMeans,0.5714,1.9912,7.2399,0,8335
13,KMeans,0.5239,1.9903,7.4232,0,1824


In [11]:
#Seleccionamos el mejor resultado
max_calinski_harabasz_score=df_clusters_selected["calinski_harabasz_score"].max()
df_cluster_selected = df_results.loc[df_results['calinski_harabasz_score'] == max_calinski_harabasz_score]
df["Cluster"]=df["TEMP_CLUSTER_"+ str(int(df_cluster_selected["n_clusters"].mean()))]
df.drop( columns=df.filter(regex='TEMP_CLUSTER_').columns,inplace= True)
print ("Cluster seleccionado: ",df_cluster_selected["n_clusters"].mean())

Cluster seleccionado:  4.0


In [12]:
# guardar data preparada
df.to_csv("data_preparada_cluster.csv")